# Car ride-share potential in mid-size U.S. cities from geographic spread

## (notebook 3: CITY_INDEX=2)

Third notebook for the IBM Data Science Specialization on Coursera, which contains the consolidated methods and functions from the first two notebooks, to serve as a template to be used per-city.



## Installs (if needed) and imports

See notesbooks 1 and 2 for details.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

try:
    from geopy.geocoders import Nominatim
    print(Nominatim)
except:
    !conda install -c conda-forge geopy=1.18.1 --yes
    
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import folium
    print(folium)
except:
    !conda install -c conda-forge folium=0.8.0 --yes
    
import folium
import time
    
try:
    import pandas_profiling
    print(pandas_profiling)
except:
    !conda install -c conda-forge pandas-profiling=1.4.1 --yes
    
import pandas_profiling

<class 'geopy.geocoders.osm.Nominatim'>
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.8.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  40.11 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  17.86 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  20.27 MB/s
folium-0.8.0-p 100% |################################| Time: 0:00:00  23.81 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    pandas-profiling: 1.4.1-py_1 conda-forge

pandas-profili 100% |####################

## Foursquare secret

In [2]:
# (paste secret from local file)



In [3]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


## Which city?

In [4]:
CITY_INDEX=2
RADIUS=800
MAX_COORDS=1000
MAP_ZOOM_LEVEL=11
MARKER_ZOOM=0.1

## All function definitions

See notebooks 1 and 2 for details.

In [5]:
print('Starting: Get list of cities')

url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })

print('---------CITIES TABLE (raw)')
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']

print('----------CITIES DATAFRAME')
print(cities_df)


def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False,
                       failsafe_abort_num=4000,
                       simple_rendering=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )
    
    num_markers=0

    # add markers to map
    for coords, venues in venues_dict.items():
        num_markers += 1
        if (venues.shape[0] > 0) and (num_markers < failsafe_abort_num): 
            if simple_rendering:
                folium.Circle(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    color=None,
                    fill=True,
                    fill_color='#005090',
                    fill_opacity=0.7).add_to(new_map)
            else:
                label = '{}, # venues: {}'.format(coords, venues.shape[0])
                label = folium.Popup(label, parse_html=True)
                folium.CircleMarker(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    popup=label,
                    color='blue',
                    fill=True,
                    fill_color='#3186cc',
                    fill_opacity=0.7,
                    parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map


def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


def gather_distance_distribution_stats(venues_agg_df, city_index=-1, city_display='(city name, state)', return_as_list_append_string=False):
    
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    num_venues_per_coord = venues_agg_df['num_venues']
    dist_per_coord = venues_agg_df['dist']
    
    # blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
    dist_list = []
    for this_coord in zip(num_venues_per_coord, dist_per_coord):
        this_list = [float(this_coord[1])] * int(this_coord[0])
        dist_list.extend(this_list)
    
    dist_list_df = pd.DataFrame(dist_list)
    dist = dist_list_df[0]
    
    # now perform statistics on that new list
    
    mean = dist.mean()
    std = dist.std()
    p25 = dist.quantile(0.25)
    median = dist.quantile(0.5)
    p75 = dist.quantile(0.75)
    max_d = dist.max()
    iqr = p75 - p25
    kurt = dist.kurtosis() # "peaky-ness"
    mad = dist.mad() # mean absolute deviation
    skew = dist.skew() # skewedness
    
    if return_as_list_append_string:
        retS = '# Create list at the beginning:\n'
        retS += '#     l = []\n\n'
        retS += 'l.append(['
        retS += '{}, "{}", ({:.2f}, {:.2f}), '.format(city_index, city_display, center_x, center_y)
        retS += '{}, {}, {}, {}, '.format(total_num_venues, total_num_indicators, mean, std)
        retS += '{}, {}, {}, {}, '.format(p25, median, p75, max_d)
        retS += '{}, {}, {}, {}'.format(iqr, kurt, mad, skew)
        retS += '])\n\n'
        retS += '# Convert list to pandas DataFrame:\n'
        retS += '#     cities_stats_df = pd.DataFrame(l)\n'
        retS += '#     cities_stats_df.columns = ['
        retS += '"city index", "city name, state", "center coord", "total num venues", "total num indicators", '
        retS += '"mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", '
        retS += '"interquartile range", "kurtosis", "mean absolute deviation", "skewedness"'
        retS += ']\n'
        return retS

    else:
        return center_x, center_y, total_num_venues, total_num_indicators, \
               mean, std, p25, median, p75, max_d,                         \
               iqr, kurt, mad, skew
    

    



Starting: Get list of cities
---------CITIES TABLE (raw)
<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></s

## Quick test

In [6]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=10, verbose=False)

print("{}, {}\n".format(city_name, city_state))

venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)

print("\n{}\n".format(venues_agg_df.head(10)))

center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew \
    = gather_distance_distribution_stats(venues_agg_df)

print(center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew)

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nlist_append_string:\n\n{}'.format(list_append_string))

make_map_from_dict(
    lat_orig, long_orig, venues_dict, 13, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)


[test #1 of 10] (0,0):35. [test #2 of 10] (-1,0):34. (1,0):41. (0,-1):53. (0,1):18. (-1,1):15. (1,-1):51. [test #3 of 10] (-1,-1):37. (0,-2):36. (1,-2):49. [test #4 of 10] (2,-1):74. (2,-2):51. [test #5 of 10] (3,-1):84. (2,0):85. (3,-2):73. [test #6 of 10] (3,0):81. (2,1):38. (1,1):30. [test #7 of 10] (4,-1):80. (4,-2):53. [test #8 of 10] (4,0):55. (3,1):33. [test #9 of 10] (5,-1):25. (5,-2):24. [test #10 of 10] (3,-3):26. (4,-3):16. Wichita, Kansas


     coord  num_venues  num_indicators
0  (5, -1)          25               1
1  (5, -2)          24               1
2   (0, 0)          35               3
3  (-1, 0)          34               1
4  (3, -3)          26               0
5   (3, 0)          81               2
6  (2, -1)          74               1
7   (2, 1)          38               1
8  (3, -2)          73               0
9  (2, -2)          51               0

2.11612364244 -0.808688387636 1197 23 1.7270626225005208 0.7959462133354515 1.1969578347156995 1.8124123106792869

## The real deal

In [7]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=MAX_COORDS, verbose=False)

print("\n\n{}, {}\n".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
print('Total shape: {}\n\nSample 10 venues per coordinate (unsorted):'.format(venues_agg_df.shape))
print('{}\n'.format(venues_agg_df.head(10)))

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nResulting list append string for this city:\n\n{}'.format(list_append_string))


[test #1 of 1000] (0,0):35. [test #2 of 1000] (-1,0):34. (1,0):41. (0,-1):53. (0,1):18. (-1,1):15. (1,-1):51. [test #3 of 1000] (-1,-1):37. (0,-2):36. (1,-2):49. [test #4 of 1000] (2,-1):74. (2,-2):51. [test #5 of 1000] (3,-1):84. (2,0):85. (3,-2):73. [test #6 of 1000] (3,0):81. (2,1):38. (1,1):30. [test #7 of 1000] (4,-1):80. (4,-2):53. [test #8 of 1000] (4,0):55. (3,1):33. [test #9 of 1000] (5,-1):25. (5,-2):24. [test #10 of 1000] (3,-3):26. (4,-3):16. [test #11 of 1000] (5,0):20. (4,1):4. [test #12 of 1000] (5,-3):8. [test #13 of 1000] (2,-3):26. [test #14 of 1000] (1,-3):28. [test #15 of 1000] [test #16 of 1000] (2,2):11. (1,2):23. [test #17 of 1000] (-2,-1):13. (-1,-2):23. (-2,0):27. [test #18 of 1000] (0,-3):21. [test #19 of 1000] (-2,1):11. [test #20 of 1000] (3,2):5. [test #21 of 1000] (0,2):17. [test #22 of 1000] (1,-4):11. (2,-4):13. [test #23 of 1000] (-3,0):8. (-3,1):6. [test #24 of 1000] (3,-4):14. (4,-4):14. [test #25 of 1000] [test #26 of 1000] (6,-1):16. (6,-2):17. [tes

9. [test #215 of 1000] (7,-12):6. (7,-13):4. [test #216 of 1000] (6,-14):7. (7,-14):4. [test #217 of 1000] (5,-14):6. [test #218 of 1000] (7,-10):17. (6,-9):16. (7,-11):14. [test #219 of 1000] (8,-10):17. (7,-9):16. (8,-11):11. [test #220 of 1000] (9,-10):19. (8,-9):9. (9,-11):9. [test #221 of 1000] (10,-10):18. (9,-9):10. (10,-11):12. [test #222 of 1000] (11,-10):13. (10,-9):12. (11,-11):12. [test #223 of 1000] (6,-8):8. (5,-8):4. [test #224 of 1000] (7,-8):4. [test #225 of 1000] (1,-6):10. [test #226 of 1000] (8,-12):9. [test #227 of 1000] (1,-10):10. (2,-11):5. [test #228 of 1000] (12,-10):11. (12,-11):8. [test #229 of 1000] (10,-12):6. (11,-12):9. [test #230 of 1000] (9,-8):14. [test #231 of 1000] (8,-8):10. (8,-7):7. [test #232 of 1000] (12,-12):8. [test #233 of 1000] [test #234 of 1000] [test #235 of 1000] (13,-9):9. [test #236 of 1000] [test #237 of 1000] (3,-13):12. (4,-14):7. [test #238 of 1000] (2,-13):12. (3,-14):13. (2,-12):8. [test #239 of 1000] (2,-14):9. (3,-15):10. (4,-

31. (-9,-2):1. [test #441 of 1000] (-9,-1):2. [test #442 of 1000] (-5,-2):26. [test #443 of 1000] (-4,-3):14. (-4,-4):14. [test #444 of 1000] (-4,-5):10. [test #445 of 1000] (-10,-3):51. (-9,-4):36. (-10,-2):7. [test #446 of 1000] (-11,-3):45. (-10,-4):51. (-11,-2):24. [test #447 of 1000] (-11,-4):39. (-10,-5):32. (-9,-5):41. [test #448 of 1000] (-12,-3):39. (-12,-2):30. [test #449 of 1000] (-8,-5):34. (-9,-6):5. (-8,-6):3. [test #450 of 1000] (-12,-4):23. (-11,-5):6. [test #451 of 1000] (-13,-3):20. (-13,-2):7. [test #452 of 1000] [test #453 of 1000] (-7,-6):3. [test #454 of 1000] (-10,-6):4. [test #455 of 1000] (-12,-1):15. (-13,-1):4. [test #456 of 1000] [test #457 of 1000] (-4,-2):6. [test #458 of 1000] (-11,-1):2. [test #459 of 1000] (-13,-4):16. (-12,-5):16. [test #460 of 1000] (-9,0):9. [test #461 of 1000] [test #462 of 1000] (-14,-3):16. (-14,-2):8. [test #463 of 1000] (-10,2):10. (-10,3):5. [test #464 of 1000] (-9,4):17. (-10,4):16. [test #465 of 1000] (-8,4):15. [test #466 of

3. (-17,-6):4. (-16,-6):7. [test #685 of 1000] [test #686 of 1000] (-20,-3):7. (-19,-4):7. (-20,-2):7. [test #687 of 1000] (-23,14):5. [test #688 of 1000] (-27,13):1. (-26,12):0. [test #689 of 1000] (-26,17):3. (-27,17):2. [test #690 of 1000] (-28,17):4. [test #691 of 1000] [test #692 of 1000] (1,7):6. (2,6):5. (1,8):7. [test #693 of 1000] (2,9):2. (1,9):1. [test #694 of 1000] (10,-14):3. [test #695 of 1000] (13,-17):4. (14,-17):5. [test #696 of 1000] (-15,-6):3. (-16,-7):3. (-15,-7):4. [test #697 of 1000] (-21,-3):17. (-20,-4):8. (-21,-2):6. [test #698 of 1000] (-22,-3):15. (-21,-4):14. (-22,-2):13. [test #699 of 1000] (-23,-3):14. (-22,-4):16. (-23,-2):13. [test #700 of 1000] (-23,-4):4. (-22,-5):2. (-21,-5):2. [test #701 of 1000] (-20,-5):2. [test #702 of 1000] (-24,-3):3. (-24,-2):4. [test #703 of 1000] (-22,-1):1. (-23,-1):2. [test #704 of 1000] (-24,-1):4. [test #705 of 1000] (-19,-5):0. [test #706 of 1000] [test #707 of 1000] (-20,-1):0. (-21,-1):3. [test #708 of 1000] (0,8):4. 

4. (27,1):4. (28,-1):5. [test #931 of 1000] [test #932 of 1000] [test #933 of 1000] (-1,13):2. (-2,12):5. (-1,12):7. [test #934 of 1000] (0,12):6. (-1,11):4. (0,11):3. [test #935 of 1000] (1,12):5. (0,13):4. (1,11):4. [test #936 of 1000] (9,-21):4. (10,-22):2. (11,-22):4. [test #937 of 1000] (17,-20):8. [test #938 of 1000] (18,-20):8. (17,-19):8. (18,-21):6. [test #939 of 1000] (19,-20):7. (18,-19):7. (19,-21):8. [test #940 of 1000] (17,-18):4. (16,-18):0. [test #941 of 1000] (20,-21):1. (19,-22):4. (20,-22):2. [test #942 of 1000] (20,-20):0. (19,-19):3. [test #943 of 1000] (18,-18):0. [test #944 of 1000] (18,-22):4. [test #945 of 1000] (13,-23):3. (14,-24):2. (15,-24):1. [test #946 of 1000] (5,-20):2. (6,-21):3. [test #947 of 1000] (7,-22):2. (8,-22):3. [test #948 of 1000] (9,-22):3. [test #949 of 1000] (-17,13):5. (-17,12):6. [test #950 of 1000] (-16,12):3. (-16,11):3. [test #951 of 1000] (27,-2):2. [test #952 of 1000] (28,-2):5. [test #953 of 1000] (29,-1):1. (29,-2):3. [test #954 o

In [8]:
make_map_from_dict(
    lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)
